In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import confusion_matrix,classification_report
from xgboost import XGBClassifier
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer



# 读取CSV文件
train_data = pd.read_csv('../data/raw/trainSet_res.csv')
train_labels = pd.read_csv('../data/raw/trainSet_ans.csv')

# 读取验证集
validation_data = pd.read_csv('../data/raw/validationSet_res.csv')

# 转换时间格式
train_data['start_time'] = pd.to_datetime(train_data['start_time'], format='%Y%m%d%H%M%S')
train_data['end_time'] = pd.to_datetime(train_data['end_time'], format='%Y%m%d%H%M%S')
train_data['open_datetime'] = pd.to_datetime(train_data['open_datetime'], format='%Y%m%d%H%M%S')
train_data['update_time'] = pd.to_datetime(train_data['update_time'])
train_data['date'] = pd.to_datetime(train_data['date'])

validation_data['start_time'] = pd.to_datetime(validation_data['start_time'], format='%Y%m%d%H%M%S')
validation_data['end_time'] = pd.to_datetime(validation_data['end_time'], format='%Y%m%d%H%M%S')
validation_data['open_datetime'] = pd.to_datetime(validation_data['open_datetime'], format='%Y%m%d%H%M%S',errors='coerce')
validation_data['update_time'] = pd.to_datetime(validation_data['update_time'])
validation_data['date'] = pd.to_datetime(validation_data['date'])

In [20]:
train_data.shape

(701815, 29)

In [21]:
# 为每条记录添加start_time_diff，记录 start_time 与上一条记录的 start_time 之差 (单位：秒)
start_time_diff = train_data.groupby('msisdn')['start_time'].diff().dt.total_seconds().fillna(0).reset_index(drop=True)
# 将该列加入到数据集中
train_data['start_time_diff'] = start_time_diff.copy()
# time_diff_start2end = train_data.groupby('msisdn')['end_time'].diff().dt.total_seconds().fillna(0)
start_time_diff = validation_data.groupby('msisdn')['start_time'].diff().dt.total_seconds().fillna(0).reset_index(drop=True)
validation_data['start_time_diff'] = start_time_diff.copy()

In [22]:
train_data.groupby('msisdn')['home_area_code'].nunique()

msisdn
1000176    1
1000184    1
1000196    1
1000238    1
1000495    1
          ..
1303634    1
1303836    1
1303962    1
1304027    1
1304080    1
Name: home_area_code, Length: 3836, dtype: int64

数据特征处理

In [23]:
# 聚合特征
def aggregate_features(data):
    return data.groupby('msisdn').agg({
    'call_duration': [
        ('call_duration_sum', 'sum'), 
        ('call_duration_mean', 'mean'), 
        ('call_duration_max', 'max'), 
        ('call_duration_quantile_25', lambda x: x.quantile(0.25)), 
        ('call_duration_quantile_50', lambda x: x.quantile(0.50)), 
        ('call_duration_quantile_75', lambda x: x.quantile(0.75))
    ],
    'cfee': [
        ('cfee_sum', 'sum'), 
        ('cfee_mean', 'mean')
    ],
    'lfee': [
        ('lfee_sum', 'sum'), 
        ('lfee_mean', 'mean')
    ],
    'hour': [
        ('hour_mean', 'mean'), 
        ('hour_std', 'std'), 
        ('hour_max', 'max'), 
        ('hour_min', 'min')
    ],
    'dayofweek': [
        ('dayofweek_std', 'std'), 
        ('magic_dayofweek', lambda x: x.value_counts().mean()), 
        ('work_day_num', lambda x: x[x.isin([1,2,3,4,5])].count()), 
        ('weekend_num', lambda x: x[x.isin([6,7])].count())
    ],
    # 'home_area_code': [
    #     ('home_area_code_nunique', 'nunique')
    # ],
    'visit_area_code': [
        ('visit_area_code_nunique', 'nunique')
    ],
    'called_home_code': [
        ('called_home_code_nunique', 'nunique')
    ],
    'called_code': [
        ('called_code_nunique', 'nunique')
    ],
    'open_datetime': [
        ('open_count', 'nunique')
    ],
    'other_party': [
        ('account_person_num', 'nunique')
    ],
    'a_serv_type': [
        ('call_num', lambda x: x[x.isin([1, 3])].count()), 
        ('called_num', lambda x: x[x == 2].count())
    ],
    'start_time_diff': [
        ('start_time_diff_mean', 'mean'), 
        ('start_time_diff_std', 'std'), 
        ('start_time_diff_max', 'max'), 
    ]
})

train_features = aggregate_features(train_data)
validation_features = aggregate_features(validation_data)


train_features.columns = ['_'.join(col).strip() for col in train_features.columns.values]
validation_features.columns = ['_'.join(col).strip() for col in validation_features.columns.values]
train_features.columns = train_features.columns.str.replace('[', '').str.replace(']', '').str.replace('<', '').str.replace('>', '').str.replace('(', '').str.replace(')', '').str.replace(',', '').str.replace(' ', '_')
validation_features.columns = validation_features.columns.str.replace('[', '').str.replace(']', '').str.replace('<', '').str.replace('>', '').str.replace('(', '').str.replace(')', '').str.replace(',', '').str.replace(' ', '_')
# 重置索引
train_features = train_features.reset_index()
validation_features = validation_features.reset_index()

# 合并标签数据
train_features = train_features.merge(train_labels, on='msisdn', how='left')
# 打印结果
train_features

,msisdn,call_duration_call_duration_sum,call_duration_call_duration_mean,call_duration_call_duration_max,call_duration_call_duration_quantile_25,call_duration_call_duration_quantile_50,call_duration_call_duration_quantile_75,cfee_cfee_sum,cfee_cfee_mean,lfee_lfee_sum,...,called_home_code_called_home_code_nunique,called_code_called_code_nunique,open_datetime_open_count,other_party_account_person_num,a_serv_type_call_num,a_serv_type_called_num,start_time_diff_start_time_diff_mean,start_time_diff_start_time_diff_std,start_time_diff_start_time_diff_max,is_sa
0,1000176,6369,68.483871,1129,20.00,37.0,78.00,0.0,0.0,0.0,...,4,1,1,44,62,31,13200.580645,24019.169180,105436.0,0
1,1000184,375,20.833333,156,3.50,8.0,13.50,0.0,0.0,0.0,...,2,2,4,6,13,5,14917.611111,23149.372122,74415.0,1
2,1000196,314,34.888889,129,6.00,12.0,43.00,0.0,0.0,0.0,...,3,3,1,6,0,9,102111.111111,162204.880678,417591.0,1
3,1000238,6882,74.804348,509,22.75,46.0,87.50,0.0,0.0,0.0,...,2,1,1,21,67,25,13530.684783,23759.086420,118769.0,0
4,1000495,218,24.222222,47,10.00,17.0,43.00,0.0,0.0,0.0,...,1,1,1,1,2,7,107095.333333,224782.881835,657421.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3831,1303634,1298,34.157895,184,17.25,24.0,34.00,0.0,0.0,0.0,...,1,1,1,2,17,21,32457.631579,56091.928599,255787.0,0
3832,1303836,10196,231.727273,1800,31.50,65.0,147.25,0.0,0.0,0.0,...,11,11,1,19,23,21,25220.590909,73156.771964,460513.0,1
3833,1303962,6524,26.958678,483,11.00,19.0,31.00,0.0,0.0,0.0,...,3,3,1,53,92,150,5157.045455,10433.356582,60155.0,0
3834,1304027,442,73.666667,254,11.50,21.0,104.75,0.0,0.0,0.0,...,3,3,1,4,3,3,80471.333333,89209.594521,212484.0,1


In [24]:
y = train_features['is_sa']
X = train_features.drop(['msisdn', 'is_sa'], axis=1)

n_sample = y.shape[0]
n_pos_sample = y[y ==1].shape[0]
n_neg_sample = y[y == 0].shape[0]
print('样本个数：{}; 正样本占{:.2%}; 负样本占{:.2%}'.format(n_sample,
                                                   n_pos_sample / n_sample,
                                                   n_neg_sample / n_sample))
print('特征维数：', X.shape[1])


样本个数：3836; 正样本占20.13%; 负样本占79.87%
特征维数： 28


In [25]:
X.head()

,call_duration_call_duration_sum,call_duration_call_duration_mean,call_duration_call_duration_max,call_duration_call_duration_quantile_25,call_duration_call_duration_quantile_50,call_duration_call_duration_quantile_75,cfee_cfee_sum,cfee_cfee_mean,lfee_lfee_sum,lfee_lfee_mean,...,visit_area_code_visit_area_code_nunique,called_home_code_called_home_code_nunique,called_code_called_code_nunique,open_datetime_open_count,other_party_account_person_num,a_serv_type_call_num,a_serv_type_called_num,start_time_diff_start_time_diff_mean,start_time_diff_start_time_diff_std,start_time_diff_start_time_diff_max
0,6369,68.483871,1129,20.00,37.0,78.0,0.0,0.0,0.0,0.0,...,1,4,1,1,44,62,31,13200.580645,24019.169180,105436.0
1,375,20.833333,156,3.50,8.0,13.5,0.0,0.0,0.0,0.0,...,1,2,2,4,6,13,5,14917.611111,23149.372122,74415.0
2,314,34.888889,129,6.00,12.0,43.0,0.0,0.0,0.0,0.0,...,1,3,3,1,6,0,9,102111.111111,162204.880678,417591.0
3,6882,74.804348,509,22.75,46.0,87.5,0.0,0.0,0.0,0.0,...,1,2,1,1,21,67,25,13530.684783,23759.086420,118769.0
4,218,24.222222,47,10.00,17.0,43.0,0.0,0.0,0.0,0.0,...,1,1,1,1,1,2,7,107095.333333,224782.881835,657421.0


In [26]:

imputer = SimpleImputer(strategy='most_frequent')
X= imputer.fit_transform(X)

X_train,X_test,y_train,y_test = train_test_split(X,y,stratify = y,test_size= 0.3,random_state=42)

# X_test_imputed = imputer.transform(X)

smote = SMOTE(random_state=42)    # 处理过采样的方法
X_train, y_train = smote.fit_resample(X_train, y_train)
print('通过SMOTE方法平衡正负样本后')
n_sample = y_train.shape[0]
n_pos_sample = y_train[y_train == 1].shape[0]
n_neg_sample = y_train[y_train == 0].shape[0]
print('样本个数：{}; 正样本占{:.2%}; 负样本占{:.2%}'.format(n_sample,
                                                   n_pos_sample / n_sample,
                                                   n_neg_sample / n_sample))
print('特征维数：', X.shape[1])


通过SMOTE方法平衡正负样本后
样本个数：4290; 正样本占50.00%; 负样本占50.00%
特征维数： 28


In [27]:
train_features.shape
columns = train_features.columns.tolist()
columns.remove('msisdn')

In [28]:
# 拼接 X_train 和 y_train np.array 为 dataframe
train_set = np.c_[X_train, y_train]
train_set = pd.DataFrame(train_set, columns=columns)
test_set = np.c_[X_test, y_test]
test_set = pd.DataFrame(test_set, columns=columns)
all_set = pd.concat([train_set, test_set], axis=0).reset_index(drop=True)

In [29]:
all_set.max()

call_duration_call_duration_sum              1.707900e+05
call_duration_call_duration_mean             1.533500e+03
call_duration_call_duration_max              2.058000e+04
call_duration_call_duration_quantile_25      6.272500e+02
call_duration_call_duration_quantile_50      1.633500e+03
call_duration_call_duration_quantile_75      2.111250e+03
cfee_cfee_sum                                3.668000e+04
cfee_cfee_mean                               8.970000e+02
lfee_lfee_sum                                6.060000e+03
lfee_lfee_mean                               3.363636e+01
hour_hour_mean                               2.300000e+01
hour_hour_std                                9.899495e+00
hour_hour_max                                2.300000e+01
hour_hour_min                                2.300000e+01
dayofweek_dayofweek_std                      3.019418e+00
dayofweek_magic_dayofweek                    4.265000e+02
dayofweek_work_day_num                       2.229000e+03
dayofweek_week

In [30]:
# 使用 autogluon 训练
from autogluon.tabular import TabularPredictor
# 输入数据X_train, y_train
model = TabularPredictor(label='is_sa', eval_metric='f1', problem_type='binary').fit(all_set, num_bag_folds=5, num_bag_sets=1, num_stack_levels=1)

No path specified. Models will be saved in: "AutogluonModels\ag-20240715_205349"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20240715_205349"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19041
CPU Count:          12
Memory Avail:     

In [31]:
print(model.evaluate(all_set))

{'f1': 0.9993693504309439, 'accuracy': 0.9994486307664032, 'balanced_accuracy': 0.9995104438642297, 'mcc': 0.9988801818546161, 'roc_auc': 0.9999999313481789, 'precision': 0.9987394957983193, 'recall': 1.0}


In [32]:
model.feature_importance(all_set)

Computing feature importance via permutation shuffling for 28 features using 5000 rows with 5 shuffle sets...
	106.39s	= Expected runtime (21.28s per shuffle set)
	31.15s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
start_time_diff_start_time_diff_max,1.486534e-02,0.001845,0.000028,5,0.018663,0.011067
called_home_code_called_home_code_nunique,1.298899e-02,0.001861,0.000049,5,0.016822,0.009156
start_time_diff_start_time_diff_std,5.446477e-03,0.001083,0.000178,5,0.007676,0.003217
call_duration_call_duration_sum,1.653895e-03,0.000444,0.000570,5,0.002569,0.000739
start_time_diff_start_time_diff_mean,1.057426e-03,0.000770,0.018647,5,0.002643,-0.000528
hour_hour_min,7.799473e-04,0.000347,0.003670,5,0.001494,0.000066
hour_hour_std,5.531883e-04,0.000419,0.020927,5,0.001416,-0.000309
dayofweek_dayofweek_std,5.038955e-04,0.000297,0.009656,5,0.001116,-0.000109
a_serv_type_called_num,1.834028e-04,0.000299,0.121247,5,0.000800,-0.000433
a_serv_type_call_num,1.374611e-04,0.000125,0.035246,5,0.000396,-0.000121


In [33]:
# leaderboard
leaderboard = model.leaderboard(all_set, silent=True)
print(leaderboard)
leaderboard

                      model  score_test  score_val eval_metric  \
0   RandomForestEntr_BAG_L1    0.999790   0.927796          f1   
1   RandomForestGini_BAG_L1    0.999790   0.925437          f1   
2     ExtraTreesEntr_BAG_L1    0.999790   0.932261          f1   
3     ExtraTreesGini_BAG_L1    0.999790   0.932823          f1   
4       WeightedEnsemble_L2    0.999790   0.940579          f1   
5     KNeighborsDist_BAG_L1    0.999790   0.857369          f1   
6     ExtraTreesGini_BAG_L2    0.999579   0.942869          f1   
7     ExtraTreesEntr_BAG_L2    0.999579   0.941624          f1   
8       WeightedEnsemble_L3    0.999369   0.944468          f1   
9   RandomForestEntr_BAG_L2    0.998949   0.943973          f1   
10  RandomForestGini_BAG_L2    0.998530   0.941626          f1   
11    KNeighborsUnif_BAG_L1    0.887452   0.842863          f1   

    pred_time_test  pred_time_val  fit_time  pred_time_test_marginal  \
0         0.094117       0.198795  1.001495                 0.094117 

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestEntr_BAG_L1,0.999790,0.927796,f1,0.094117,0.198795,1.001495,0.094117,0.198795,1.001495,1,True,4
1,RandomForestGini_BAG_L1,0.999790,0.925437,f1,0.105095,0.195034,0.864308,0.105095,0.195034,0.864308,1,True,3
2,ExtraTreesEntr_BAG_L1,0.999790,0.932261,f1,0.125045,0.216675,0.677775,0.125045,0.216675,0.677775,1,True,6
3,ExtraTreesGini_BAG_L1,0.999790,0.932823,f1,0.126293,0.223436,0.631652,0.126293,0.223436,0.631652,1,True,5
4,WeightedEnsemble_L2,0.999790,0.940579,f1,0.397014,0.699472,4.633432,0.002594,0.014573,2.314290,2,True,7
5,KNeighborsDist_BAG_L1,0.999790,0.857369,f1,0.048965,0.045993,0.008220,0.048965,0.045993,0.008220,1,True,2
6,ExtraTreesGini_BAG_L2,0.999579,0.942869,f1,0.643446,1.132972,3.839579,0.093115,0.209249,0.647115,2,True,10
7,ExtraTreesEntr_BAG_L2,0.999579,0.941624,f1,0.659247,1.122811,3.937064,0.108916,0.199088,0.744600,2,True,11
8,WeightedEnsemble_L3,0.999369,0.944468,f1,0.738965,1.333455,8.387410,0.002112,0.010261,3.675358,3,True,12
9,RandomForestEntr_BAG_L2,0.998949,0.943973,f1,0.643738,1.113945,4.064937,0.093407,0.190222,0.872473,2,True,9


In [34]:
# 在testset 上计算指标
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

y_pred = model.predict(all_set)
y_true = all_set['is_sa']
print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      3064
         1.0       1.00      1.00      1.00      2377

    accuracy                           1.00      5441
   macro avg       1.00      1.00      1.00      5441
weighted avg       1.00      1.00      1.00      5441

[[3061    3]
 [   0 2377]]


In [35]:
X_validation = validation_features.drop(['msisdn'], axis=1)

In [36]:
# 预测
y_validation_pred = model.predict(X_validation)

# 将预测结果与 msisdn 对应起来
validation_results = validation_features[['msisdn']].copy()
validation_results['is_sa'] = y_validation_pred.astype(int)

print(validation_results.describe())

# 保存结果到CSV文件
validation_results.to_csv('./vaild_large_data.csv', index=False)
print("Validation predictions saved to validation_predictions.csv")

             msisdn        is_sa
count  1.278000e+03  1278.000000
mean   1.148894e+06     0.176056
std    8.774882e+04     0.381017
min    1.000168e+06     0.000000
25%    1.073945e+06     0.000000
50%    1.148652e+06     0.000000
75%    1.226613e+06     0.000000
max    1.303594e+06     1.000000
Validation predictions saved to validation_predictions.csv
